<a href="https://colab.research.google.com/github/GioLomia/Santander_Competition/blob/master/Santander_magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Light Gradient Boosting With Imbalanced Anonymous Data Set. (Santander)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


import os
import warnings
warnings.filterwarnings('ignore')

In [0]:
##############KAGGLE ACCESS#################
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!ls
!kaggle competitions list

In [0]:
!kaggle competitions files -c santander-customer-transaction-prediction
!ls
!kaggle competitions download  -c santander-customer-transaction-prediction -p /content/kaggle
!mkdir root
%cd kaggle
!ls


In [0]:
!unzip /content/kaggle/test.csv.zip
!unzip /content/kaggle/train.csv.zip
!ls

In [0]:
train_df = pd.read_csv("/content/kaggle/train.csv")
test_df = pd.read_csv("/content/kaggle/test.csv")

In [0]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [0]:
#Model Parameters.
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.33,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.015,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 100,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 15,
    'num_threads': 120,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1
}


num_round = 100000
folds = StratifiedKFold(n_splits=12, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

In [0]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 2500)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [0]:
bst = lgb.Booster(model_file='san_magic_full_12_fold')
bst

In [0]:
bst.save_model("san_magic_full_12_fold")